# GPT API: Function Calling

Unfortunately, since the GPT 3.5 and 4 models are trained with data from June-September 2021 and December 2023 and before, they cannot know information that emerges or is discovered after June-September 2021 or December 2023. They are also inadequate for solving complex math problems. In this case, our models can be used by calling the function we define for the model to return more consistent and accurate results on topics they are unfamiliar or weak in.

For example, let's consider that the calculation of body mass index was discovered after April 2023, or the models are very bad at calculating body mass index. Let's create a function below for the model to calculate BMI correctly and then see how we call this function in GPT models.

Function calling is also very useful for pulling the data we want from a text. We will see an example in the Langchain tutorial.

In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('openai_key')

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']
)

In [ ]:
def body_mass_index(height,  weight):
  "calculate the body mass index"
  b_m_i=weight/(height*height)

  return f"The body mass index is about {b_m_i:.2f}"

# First, we define our function to calculate the body mass index.

In [ ]:
body_mass_index_func = {
    "name": "body_mass_index",
    "description": "Calculates the body mass index.",
    "parameters": {
        "type": "object",
        "properties": {
            "height": {
                "type": "number",
                "description": "person's height in meters"
            },
            "weight": {
                "type": "number",
                "description": "person's weight in kilograms"
            }

        },
        "required": ["height",  "weight"]
    }
}

# We define the function name, description (intended use), function parameters and type as shown above. 
# The most important part here is the description. Because it is based on these function descriptions that the model decides whether to call a function or not.
# Also, based on the parameter descriptions, the model can select the relevant parameters from the text.

# If the question is semantically very close to the function description, the model will call that function. However, if the function description is poorly done or incomplete, the model will not call the function.

# After the model calls the function, it is important to have well-defined parameter descriptions to select the correct parameters from the text.
# If the parameter descriptions are poorly done or missing, the model will not be able to select the correct parameters from the text.

In [ ]:
type(body_mass_index_func)

dict

In [ ]:
prompt = "What is the body mass index of a person who is 1800 mm tall and weight 80000 grams?"

res = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": prompt}],
    functions=[body_mass_index_func],
    function_call="auto"
)
res

# Set the 'functions' parameter to 'body_mass_index_func'.

# Set the 'function_call' parameter to 'auto'.

ChatCompletion(id='chatcmpl-9Gt5ubEnkTvt0nAEEDrCn1J3CmVgI', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"height":1.8,"weight":80}', name='body_mass_index'), tool_calls=None))], created=1713812058, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=21, prompt_tokens=87, total_tokens=108))

In [ ]:
res.choices[0].message.function_call.arguments
# We extract the selected parameters from the Prompt text.

# As you can see, our output is in JSON format (string).
# In Python, we cannot use JSON format data directly. Therefore, we need to convert the data to dictionary format using the json.loads() function
# In Python we can use data in JSON format.
# Once the data is converted to dictionary format, we can easily extract height and weight information.

'{"height":1.8,"weight":80}'

In [ ]:
import json

name = res.choices[0].message.function_call.name
arguments = json.loads(res.choices[0].message.function_call.arguments) # Use the json.loads() function to translate a JSON string into a dictionary.
                                                                       # This function translates a JSON string into a dictionary.
print(name)
print(arguments)

body_mass_index
{'height': 1.8, 'weight': 80}


In [ ]:
print(arguments["height"])
print(arguments["weight"])

# We were able to easily capture height and weight information

1.8
80


In [ ]:
body_mass_index(arguments["height"],  arguments["weight"])

# When I write the parameters into the function we defined, it returns the output I want.

'The body mass index is about 24.69'

In [ ]:
def run_conversation(prompt):

    response = client.chat.completions.create(model='gpt-3.5-turbo',
                                              messages=[{"role": "user", "content": prompt}],
                                              functions=[body_mass_index_func],
                                              function_call="auto")


    # Step 1, check if the model wants to call a function (function_calling)
    if  response.choices[0].finish_reason == "function_call":
        # convert json string to dictionary
        arguments = json.loads(response.choices[0].message.function_call.arguments)

        # Step 2, call the function and assign it to a variable
        function_response = body_mass_index(arguments["height"],
                                            arguments["weight"])

        return function_response # return the result of the function
    return response.choices[0].message.content # if the function is not called. Return the kenid response of chatgpt.


# If the model returns "function_call" as "finish_reason" after the text we give to the run_conversation function,
# will give its answers via the body_mass_index function. Otherwise the model will give its answers in its own way.

In [ ]:
run_conversation("What is the body mass index of a person who is 1800 mm tall and weight 80000 grams?")

'The body mass index is about 24.69'

In [ ]:
run_conversation("What is the Capital of Spain?")

'The capital of Spain is Madrid.'

END OF THE PROJECT